In [1]:
%load_ext Cython

In [10]:
import numpy as np
import pandas as pd
import time

def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        x_val=data.loc[i, x_name]
        y_val=data.loc[i, y_name]
        if x_val not in value_dict.keys():
            value_dict[x_val] = y_val
            count_dict[x_val] = 1
        else:
            value_dict[x_val] += y_val
            count_dict[x_val] += 1
    for i in range(data.shape[0]):
        val_loc=data.loc[i, x_name]
        result[i] = (value_dict[val_loc] - data.loc[i, y_name]) / (count_dict[val_loc] - 1)
    return result

y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

In [9]:
%%timeit
target_mean_v2(data,'y','x')

143 ms ± 6.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


cpdef func and some parameters

In [12]:
%%cython -a
import numpy as np
import pandas as pd
import time
cimport numpy as np
import cython
cimport cython
from cython.parallel import prange

cpdef convert_demo(matrix,num_dim=1):
    cdef np.ndarray[double, ndim=1, mode='fortran'] arg = np.asfortranarray(matrix, dtype=np.float64)
    return arg


cpdef target_mean_v3(data, y_name, x_name):                  
    result = convert_demo(np.zeros(data.shape[0]))
    value_dict = dict()
    count_dict = dict()
    
    cdef x_val,y_val,i,val_loc
    
    for i in range(data.shape[0]):
        x_val=data.loc[i, x_name]
        y_val=data.loc[i, y_name]
        if x_val not in value_dict.keys():
            value_dict[x_val] = y_val
            count_dict[x_val] = 1
        else:
            value_dict[x_val] += y_val
            count_dict[x_val] += 1
    for i in range(data.shape[0]):
        val_loc=data.loc[i, x_name]
        result[i] = (value_dict[val_loc] - data.loc[i, y_name]) / (count_dict[val_loc] - 1)
    return result



In [14]:
%%timeit
target_mean_v3(data,'y','x')

136 ms ± 5.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


magic func and cdef parameters

In [15]:
%%cython -a
import numpy as np
import pandas as pd
import time
cimport numpy as np
import cython
cimport cython
from cython.parallel import prange

cpdef target_mean_v5(long[:,:] data):                       
    cdef double[:] result = np.zeros(data.shape[0])
    cdef dict value_dict={}
    cdef dict count_dict={}
    cdef int x_val
    cdef int y_val
    cdef int i
    cdef int val_loc
    
    for i in range(data.shape[0]):
        x_val=data[i][1]
        y_val=data[i][0]
        if x_val not in value_dict.keys():
            value_dict[x_val] = y_val
            count_dict[x_val] = 1
        else:
            value_dict[x_val] += y_val
            count_dict[x_val] += 1
    for i in range(data.shape[0]):
        val_loc=data[i][1]
        result[i] = (value_dict[val_loc] - data[i][0]) / (count_dict[val_loc] - 1)
    return result

y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data0=np.concatenate([y, x], axis=1)

In [16]:
%%timeit
target_mean_v5(data0)

1.06 ms ± 40.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


prange&nogil

In [18]:
%%cython -a
import numpy as np
import time
cimport numpy as np
cimport cython
from cython.parallel import prange

cpdef target_mean_v6(np.ndarray[long,ndim=2] data):    #prange,nogil
    cdef int num=data.shape[0]
    cdef np.ndarray[double,ndim=1] result = np.zeros(num)
    cdef np.ndarray[double,ndim=1] value_dict=np.zeros(10)
    cdef np.ndarray[double,ndim=1] count_dict=np.zeros(10)
    cdef int x_val,y_val,i,val_loc
    cdef np.ndarray[long,ndim=1] y_value=data[:,0]
    cdef np.ndarray[long,ndim=1] x_value=data[:,1]
    
    for i in prange(num,nogil=True):
        value_dict[x_value[i]] += y_value[i]
        count_dict[x_value[i]] += 1
        
    for i in prange(num,nogil=True):
        val_loc=x_value[i]
        result[i] = (value_dict[val_loc] - y_value[i]) / (count_dict[val_loc] - 1)
    return result

In [19]:
%%timeit
target_mean_v6(data0)

57.3 µs ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


The spend times summarize:

143 ms ± 6.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

136 ms ± 5.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

1.06 ms ± 40.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

57.3 µs ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
